In [28]:
import pandas as pd

In [29]:
from dotenv import load_dotenv
import os 
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
load_dotenv('.env')


True

In [30]:
model = ChatOpenAI(model="gpt-4o")
model.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BWf3mAAPyR7dz71laFJgUAhvsbVyU', 'finish_reason': 'stop', 'logprobs': None}, id='run-caafb2c1-d310-4eca-ae7e-618acf5ea66d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Qdrant implementation

In [31]:
csv_file = "data/hotel_with_nearest_restaurants.csv"
df = pd.read_csv(csv_file)
df

,hotel_id,hotel_name,total_rooms,admin,hotel_price,city,address,map_link,hotel_features,short_description,long_description,latitude,longitude,nearest_restaurants,link_web
0,1,DeLaSea Phát Linh Hạ Long,326,CÔNG TY CỔ PHẦN ĐẦU TƯ TÂN HOA LƯ,1950000,Hạ Long,"Lô 1, Đại Lộ Hạ Long Marine, Phường Bãi Cháy, ...",https://maps.app.goo.gl/yxDCjgQvorHL43xX9,"Phòng có bồn tắm, Lễ tân 24h, Wifi miễn phí, N...","Với tông màu chủ đạo vàng, trắng sử dụng trong...",Khách sạn 5 sao Phát Linh tọa lạc tại đại lộ M...,20.955721,106.935051,"Nhà hàng: Ăn Đêm Đồ Nhậu, Địa chỉ: Quốc Lộ 18 ...",http://localhost/khach-san/1
1,2,Citadines Marina Hạ Long,580,The Ascott Limited,2000000,Hạ Long,"Bán đảo 3, Marina Halong, Hạ Long, Quảng Ninh",https://maps.app.goo.gl/yseMCyJmvGiZ1t7v9,"Wifi miễn phí, Lễ tân 24h, Bể bơi ngoài trời, ...",Sở hữu vị trí trung tâm và tọa lạc ngay gần co...,Citadines Marina Halong toạ lạc tại tổ hợp du ...,20.953638,106.930797,"Nhà hàng: Ăn Đêm Đồ Nhậu, Địa chỉ: Quốc Lộ 18 ...",http://localhost/khach-san/2
2,3,Sea Stars Hotel Hạ Long,354,Sea Stars Group,2700000,Hạ Long,"Lô N8-1/Số 47 Cái Dăm, Khu đô thị, Thành phố H...",https://maps.app.goo.gl/8EyutG7DHVyU4Xf88,"Bể bơi ngoài trời, Wifi miễn phí, Ban công/Cửa...","Tại Sea Stars, chúng tôi cung cấp đầy đủ những...",Sea Stars Hotel Hạ Long tự hào là khách sạn 5 ...,20.948831,106.941587,"Nhà hàng: Nhà Hàng Quỳnh Ngọc, Địa chỉ: 574,Qu...",http://localhost/khach-san/3
3,4,Wyndham Legend Hạ Long,450,CÔNG TY CỔ PHẦN DU LỊCH DỊCH VỤ SAO HẠ LONG,2500000,Hạ Long,"12 Hạ Long, Bãi Cháy, Thành phố Hạ Long, Quảng...",https://maps.app.goo.gl/A5ueMVJi4M7CXhJG7,"Phòng có bồn tắm, Lễ tân 24h, Wifi miễn phí, B...",Nhà hàng Blue Bay không chỉ nổi bật với thực đ...,Wyndham Legend Hạ Long là khách sạn 5 sao đầu ...,20.959447,106.977972,"Nhà hàng: Nhà Hàng Vân Anh, Địa chỉ: Quốc Lộ 1...",http://localhost/khach-san/4
4,5,Hạ Long Dream Hotel,186,Geleximco Group,1500000,Hạ Long,"số 10 đường Hạ Long, phường Bãi Cháy, Tp. Hạ Long",https://maps.app.goo.gl/EMV8uhrwSEetkPX17,"Phòng có bồn tắm, Lễ tân 24h, Wifi miễn phí, N...","Các phòng nghỉ với phong cách thiết kế mới mẻ,...",Khách sạn Ha Long Dream là một trong những khá...,20.956582,106.967884,"Nhà hàng: Nhà Hàng Quỳnh Ngọc, Địa chỉ: 574,Qu...",http://localhost/khach-san/5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,215,Melia Vinpearl Hotel Huế,213,Melia Vinpearl Hotel Huế,2500000,Thừa Thiên Huế,"50A, đường Hùng Vương, TP.Huế, tỉnh Thừa Thiên...",https://maps.app.goo.gl/N47jGfG8f3ehgj7V9,"Wifi miễn phí, Phòng có bồn tắm, Nhà hàng, Lễ ...",Bể bơi bốn mùa. Hệ thống nhà hàng và bar: Nhà ...,Melia Vinpearl Hotel Huế - nơi truyền thống th...,16.463047,107.591816,"Nhà hàng: Vuon Pho Restaurant, Địa chỉ: 6A Ly ...",http://localhost/khach-san/215
215,216,Mường Thanh Grand Gia Lai,160,Mường Thanh Grand Gia Lai,1500000,Gia Lai,"537 Hùng Vương, Phường Phù Đổng, Thành phố Pl...",https://maps.app.goo.gl/BpN9cYfvMQXvWrCTA,"Wifi miễn phí, Phòng có bồn tắm, Nhà hàng, Lễ ...",Tọa lạc ngay trung tâm thành phố Pleiku Các ph...,"Mường Thanh Grand Gia Lai Hotel, khách sạn 4 s...",13.972530,108.011678,"Nhà hàng: Quán Nhậu Hai Cảng, Địa chỉ: 1 Nguyễ...",http://localhost/khach-san/216
216,217,Golden Boutique Hotel Măng Đen,62,Golden Boutique Hotel Măng Đen,2300000,Kon Tum,"Lô biệt thự số 26 Nguyễn Du, Thị Trấn Măng Đen...",https://maps.app.goo.gl/4FYFbig8uhjF539HA,"Bể bơi ngoài trời, Phòng có bồn tắm, Wifi miễn...",Tọa lạc giữa trung tâm cao nguyên Măng Đen Hồ ...,Golden Boutique Hotel Măng Đen Kon Tum&nbsp;tọ...,14.585493,108.278374,Nhà hàng: Gà Nướng Cơm Lam Cô Sinh ( Chính H...,http://localhost/khach-san/217
217,218,Stella Maris Beach Đà Nẵng,97,Stella Maris Beach Đà Nẵng,1500000,Bà Rịa - Vũng Tàu,"03 Võ Văn Kiệt, Phường Phước Mỹ, Quận Sơn Trà...",https://maps.app.goo.gl/KMJ4EwRrARtaToPMA,"Bể bơi ngoài trời, Wifi miễn phí, Phòng có bồn...",Nằm cách bãi biển Mỹ Khê chỉ 50m Hồ bơi trên t...,Khách sạn Stella Maris Beach Đà Nẵng được thiế...,16.063502,108.242

In [32]:
from langchain.schema import Document

doc = []
ship_file = "data/ship.csv"
df3 = pd.read_csv(ship_file)
for idx, row in df3.iterrows():
    doc.append(Document(page_content=f"Tàu {row['ship_name']} có link trên web MonkeyDvuvi là {row['link_web']}, có số cabin là {row['cabin']}, trip đi {row['trip']} bởi công ty {row['admin']}, giá thành là {row['ship_price']}, địa chỉ ở {row['address']} với link map {row['map_link']}. Thuyền gồm các tiện ích sau {row['ship_features']}, {row['long_description']}\n", metadata={"type": "ship"}))
    
room_ship_file = "data/ship_rooms.csv"
df4 = pd.read_csv(room_ship_file)
df4 = df4.merge(df3[['ship_id', 'ship_name']], on='ship_id', how='left')

for idx, row in df4.iterrows():
    doc.append(Document(page_content=f"Tàu {row['ship_id']} có phòng loại {row['room_name']}, có kích thước {row['size']}m2, chứa được {row['max_persons']}. Giá phòng là {row['room_price']} và có các đặc trưng sau {row['room_features']}\n", metadata={"type": "ship_room", "ship_name": row['ship_name']}))
for idx, row in df.iterrows():
    doc.append(Document(page_content=f"Khách sạn {row['hotel_name']} có link trên web MonkeyDvuvi là {row['link_web']} với tổng cộng {row['total_rooms']} phòng, được quản lý bởi {row['admin']}, có giá {row['hotel_price']} đồng, nằm tại {row['city']}, địa chỉ {row['address']}, với đường dẫn bản đồ {row['map_link']}, có các tiện ích {row['hotel_features']}, và có các nhà hàng gần nhất:\n{row['nearest_restaurants']}, mô tả ngắn gọn: {row['short_description']}, mô tả chi tiết: {row['long_description']} \n", metadata={"city": row['city'], "type": "hotel"}))
df2 = pd.read_csv("data/restaurant_final.csv")
for idx, row in df2.iterrows():
    doc.append(Document(page_content=f"Nhà hàng {row['name']} có địa chỉ {row['address']} với link Map {row['map_link']} có đánh giá {row['rating']}, giờ mở cửa là {row['open_hours']}, website {row['website']}, và số điện thoại là {row['phone']} \n", metadata={"type": "restaurant"}))

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50)
split_docs = splitter.split_documents(doc)

embeddings = OpenAIEmbeddings()


In [34]:
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Start Qdrant locally or use cloud, then connect
client = QdrantClient(host="localhost", port=6333)

# Create collection
client.recreate_collection(
    collection_name="hotels_and_ship_and_restaurants",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

# Index documents
vectorstore = Qdrant.from_documents(
    documents=split_docs,
    embedding=embeddings,
    location="http://localhost:6333",
    collection_name="hotels_and_ship_and_restaurants"
)


C:\Users\ADmin\AppData\Local\Temp\ipykernel_20304\4072912840.py:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


### Reloading vector database

In [14]:
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain.embeddings import OpenAIEmbeddings

# Kết nối lại Qdrant
client = QdrantClient(host="localhost", port=6333)
embeddings = OpenAIEmbeddings()

# Load vectorstore từ collection đã tồn tại
vectorstore = Qdrant(
    client=client,
    collection_name="hotels_and_ship_and_restaurants",
    embeddings=embeddings
)

C:\Users\ADmin\AppData\Local\Temp\ipykernel_35644\87818345.py:10: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


In [15]:
# Gọi truy vấn
results = vectorstore.similarity_search("Xây dựng một tour đi Tràng An và các điểm liên quan 2 ngày 1 đêm", k=10)

In [15]:
prompt_template = """
Bạn là một chuyên gia du lịch và có kiến thức đầy đủ về các khách sạn tại Việt Nam. Bạn hãy trả lời câu hỏi của tôi dựa trên thông tin mà tôi đã cung cấp cho bạn. Các câu hỏi chủ yếu về các địa điểm du lịch, và cách xây dựng tour du lịch hợp lí bao gồm khách sạn, nhà hàng cũng như các địa điểm du lịch nổi tiếng gần đó.
Dưới đây là thông tin về một khách sạn mà tôi đã tìm thấy: {results}

Câu hỏi của tôi là: {question}

Hãy cung cấp thông tin chi tiết về khách sạn và nhà hàng mà tôi yêu cầu, sử dụng các thông tin đã có và bổ sung thêm nếu có. Nếu có các thông tin về đường dẫn tới map hay facebook, website của khách sạn hay nhà hàng, hãy cung cấp cho tôi. Nếu không có thông tin nào, hãy nói là không có thông tin nào.
"""

In [16]:
messages = model.invoke(prompt_template.format(results=results, question="Sửa gợi ý bên trên, cho thông tin khách sạn lên trước tiên"))
print(messages.content)

Dưới đây là thông tin chi tiết về khách sạn và nhà hàng mà bạn yêu cầu:

### Khách sạn Tam Coc Lion Kings Hotel & Resort, Ninh Bình
- **Địa chỉ**: Ninh Bình, cách Chùa Bái Đính 27 km.
- **Tiện ích**: Cung cấp xe đạp miễn phí, chỗ đậu xe riêng miễn phí, hồ bơi ngoài trời và khu vườn. Khách sạn có Wi-Fi miễn phí trong các khu vực công cộng.
- **Khoảng cách đến các địa điểm nổi tiếng**:
  - Cách Tam Cốc Ninh Bình 1.9 km.
  - Cách Đền Thung Nắng không xa.
  - Cách Boat trip 5 phút đi bộ.
  - Cách Phim Trường Kong Skull Island một khoảng cách đi bộ ngắn.
- **Sân bay gần nhất**: Cách sân bay Thanh Hóa Thọ Xuân 60 km.
- **Bữa sáng**: Phục vụ bữa sáng kiểu lục địa với nhiều món ăn khác nhau.
- **Nhà hàng gần nhất**: Nhà hàng Tam Cốc Ninh Bình và Hang Mua Viewpoint cách khách sạn 20 phút đi bộ.

### Nhà hàng Ba Cửa
- **Địa chỉ**: Tràng An Truong Yen 430000, Ninh Bình.
- **Khoảng cách từ khách sạn**: 9.8 km.
- **Số điện thoại**: 091 631 16 58.
- **Link Google Maps**: [Nhà Hàng Ba Cửa](https://ww

In [28]:
messages.content

'Dưới đây là gợi ý cho một tour du lịch 2 ngày 1 đêm tại Ninh Bình, bao gồm thông tin về khách sạn, nhà hàng và các địa điểm du lịch nổi tiếng:\n\n### Ngày 1:\n- **Sáng:**\n  - Khởi hành từ Hà Nội đến Ninh Bình (khoảng 2 giờ đi xe).\n  - Tham quan **Khu du lịch Tràng An**, một trong những di sản thế giới được UNESCO công nhận. Bạn có thể tham gia tour thuyền để khám phá các hang động và cảnh quan thiên nhiên tuyệt đẹp.\n\n- **Trưa:**\n  - Ăn trưa tại **Nhà Hàng Hương Sen**.\n    - Địa chỉ: Hoa Lư - Ninh Bình, Ninh Bình.\n    - Khoảng cách từ Tràng An: 6.9 km.\n    - [Link Google Maps](https://www.bing.com/maps?cp=20.283302307128906%7E105.89414978027344&lvl=17.0&q=Nh%C3%A0%20H%C3%A0ng%20H%C6%B0%C6%A1ng%20Sen%20%2C%20Hoa%20L%C6%B0%20-%20Ninh%20B%C3%ACnh%20Ninh%20B%C3%ACnh%20Ninh%20B%C3%ACnh)\n    - Website: [Facebook](https://www.facebook.com/people/Nh%C3%A0-H%C3%A0ng-H%C6%B0%C6%A1ng-Sen/100066761656887/)\n    - Phone: 097 752 54 70\n\n- **Chiều:**\n  - Tham quan **Chùa Bái Đính**, ngôi 

In [11]:
from langchain_community.tools import TavilySearchResults

tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("Gợi ý tour du lịch Tràng An 2 ngày 1 đêm")

     # Format
formatted_search_docs = "\n\n---\n\n".join(
    [
        f'<Document href="{doc["url"]}">\n{doc["content"]}\n</Document>'
        for doc in search_docs
    ]
)
print(formatted_search_docs)

<Document href="https://eholiday.vn/tour-trang-an-bai-dinh-2-ngay-1-dem/">
Tour Tràng An Bái Đính 2 ngày 1 đêm sẽ đưa quý khách đi du ngoạn những địa danh đẹp và nổi tiếng nhất tại Ninh Bình như: Chùa Bái Đính - ngôi chùa được mệnh
</Document>

---

<Document href="https://thesinhtour.com/tour-ninh-binh-2-ngay-1-dem-tu-ha-noi/?srsltid=AfmBOoqcFwtC6Mw-Y2khQbmPc5C8QvTy7SMnlbO4umWmfPyjX17vmz6Z">
Với tuyến này quý khách có cơ hội tham quan 4 hang động tự nhiên trong số những hang động đẹp nhất trong khu du lịch Tràng An là Hang Lấm, Hang Vạng, Hang Thánh Trượt và hang cuối cùng là Hang Đại. Tuyến này sẽ rất phù hợp với những bạn trẻ, những người thích chụp ảnh Check in, quý khách có thể có những trải nghiệm đầy đủ về hành trình ngồi đò thăm hang cũng như có thể lưu lại những cảm xúc, những bức hình đẹp trong hành trình về thăm Tràng An – Di sản văn hòa và thiên nhiên thế giới! [...] 12h00| Quý khách về nhà hàng nghỉ ngơi ăn trưa, cùng thưởng thức các đặc sản nơi đây (Cơm cháy, dê núi…)

13